In [1]:
import google.generativeai as genai
import google.generativeai as genai
import ast
import os
import random
from nest_asyncio import apply
from templates import templates
import json
from image_agent import get_images
from youtube import get_data
import asyncio
apply()

# Load environment variables from a .env file
from dotenv import load_dotenv

load_dotenv()

from wikipedia_tool import wikipedia_tool

# Parse API keys stored in an environment variable and convert them into a Python list
GEMINI_API_KEYS = os.environ.get("GEMINI_API_KEYS")
KEY_LIST = ast.literal_eval(GEMINI_API_KEYS)

# Shuffle the API keys list to ensure usage of different keys over time
random.shuffle(KEY_LIST)

# Initialize a global index to track the current API key being used
current_api_key_index = 0

genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))


# List to store historical messages for reference or logging
messages = []


def cycle_api_key():
    """Retrieve the next API key from the list, cycling back to the start if necessary."""
    global current_api_key_index
    if current_api_key_index >= len(KEY_LIST) - 1:
        current_api_key_index = 0
    else:
        current_api_key_index += 1
    return KEY_LIST[current_api_key_index]


def generate_new_model():
    """Generate and configure a new AI model instance with a cycled API key."""
    global current_api_key_index
    global messages
    api_key = cycle_api_key()  # Cycle to the next API key

    # Configure the generative AI model with the new API key
    genai.configure(api_key=api_key)

    # Initialize the model with specific configurations
    model = genai.GenerativeModel(
        "gemini-1.5-pro-latest",
        generation_config=genai.GenerationConfig(
            temperature=0,  # Set deterministic behavior for the model
            max_output_tokens=8000
        ),
    )
    return model

/Users/artemis/miniconda3/envs/teachme/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
topic = "stacks in programing"

sources = wikipedia_tool.run(topic)
print(sources)

Page: Stack (abstract data type)
Summary: In computer science, a stack is an abstract data type that serves as a collection of elements with two main operations:

Push, which adds an element to the collection, and
Pop, which removes the most recently added element.Additionally, a peek operation can, without modifying the stack, return the value of the last element added. The name stack is an analogy to a set of physical items stacked one atop another, such as a stack of plates. 
The order in which an element added to or removed from a stack is described as last in, first out, referred to by the acronym LIFO. As with a stack of physical objects, this structure makes it easy to take an item off the top of the stack, but accessing a datum deeper in the stack may require removing multiple other items first.Considered a linear data structure, or more abstractly a sequential collection, a stack has one end which is the only position at which the push and pop operations may occur, the top of 

In [3]:
model = generate_new_model()
audio, video = get_data(topic)

Video downloaded to /Users/artemis/Documents/Hackathons/TeachMe/server/Experiments/videos/video.mp4
Audio downloaded to /Users/artemis/Documents/Hackathons/TeachMe/server/Experiments/videos/audio.mp3
Extracting /Users/artemis/Documents/Hackathons/TeachMe/server/Experiments/videos/video.mp4 at 1 frame per second. This might take a bit...
Completed video frame extraction!

Extracted: 52 frames
Uploading 52 files. This might take a bit...
Uploading: content/frames/video/vid_frame00_00.jpg...
Uploading: content/frames/video/vid_frame00_01.jpg...
Uploading: content/frames/video/vid_frame00_02.jpg...
Uploading: content/frames/video/vid_frame00_03.jpg...
Uploading: content/frames/video/vid_frame00_04.jpg...
Uploading: content/frames/video/vid_frame00_05.jpg...
Uploading: content/frames/video/vid_frame00_06.jpg...
Uploading: content/frames/video/vid_frame00_07.jpg...
Uploading: content/frames/video/vid_frame00_08.jpg...
Uploading: content/frames/video/vid_frame00_09.jpg...
Uploading: content/f

In [4]:
def sources_to_lecture(original_prompt, sources, audio, video):
    """Converts a list of sources into a single lecture template.

    Args:
        sources (list): A list of sources, each containing a 'content' key with the source content.

    Returns:
        str: A single lecture template combining all the source content.
    """
    prompt = (
        "Available templates: \n\n"
        + str(templates)
        + "\n\n"
        + "Original Topic: "
        + original_prompt
        + "\n\n"
        + """
You are an advanced assistant that is in charge of aggregating multiple sources of information into a lecture based on a specific prompt.
Output six different slides on the topic given above using the sources provided as well as the given templates.
Make sure the slides flow logically and are easy to understand. Use the correct templates for the content you are presenting. 
Keep text content short and concise, at most 8 words per text.
Make sure you always start off with a title slide.

Try to make each slide have different information.

Return a json parsable string of lectures that are generated from the sources provided.
Make sure the response is in the following format:
{
    "title": title of the lecture,
    "description" : description of the lecture,
    "slides": [
        {
        "title": title of the slide,
        "template_id": 1
        "texts" : [
            "a stack is a data structure",
            "stacks are used in DFS"
        ]
        "speaker_notes" : things that are not on the slide but are important to mention
        }   
    ]
}
"""
    )
    lecture = model.generate_content(
        video
        + [
            audio,
            sources + prompt,
        ],
        request_options={"timeout": 1000},
    )
    return lecture.text

result = sources_to_lecture(topic, sources, audio, video)
if "```json" in result:
    # Get the JSON content from the result
    result = result.split("```json")[1]
    result = result.split("```")[0]

result = json.loads(result)
print(result)

{'title': 'Stacks in Programming', 'description': 'An exploration of stacks, their properties, and applications in programming.', 'slides': [{'title': 'Stacks in Programming', 'template_id': 4, 'texts': [], 'speaker_notes': 'Welcome to the lecture on stacks in programming. Today, we will delve into the world of stacks, exploring their fundamental characteristics, operations, and diverse applications in the realm of computer science.'}, {'title': 'What is a Stack?', 'template_id': 0, 'texts': ['LIFO data structure', 'Push and pop operations', 'Like a stack of plates'], 'image': 'stack_image.jpg', 'speaker_notes': "A stack is a Last-In-First-Out (LIFO) data structure, where the last element added is the first one to be removed. Imagine a stack of plates: you can only add or remove plates from the top. Stacks support two primary operations: 'push' to add an element and 'pop' to remove the top element."}, {'title': 'Stack Operations', 'template_id': 9, 'texts': ['Push: Adds an element', 'P

In [6]:
tasks = []
new_slides = []

for slide in result['slides']:
    template = [t for t in templates if t['template_id'] == slide['template_id']][0]
    num_images = template['num_images']
    
    if num_images == 0:
        new_slides.append({
            **slide,
            "images": []
        })
    else:
        topic = slide['speaker_notes']
        print(topic)
        # Schedule the get_images task for concurrent execution
        task = get_images(topic, num_images)
        tasks.append(task)

# Run all tasks concurrently and collect results
images_results = await asyncio.gather(*tasks)

# Iterate over the slides that require images
image_index = 0
for slide in result['slides']:
    template = [t for t in templates if t['template_id'] == slide['template_id']][0]
    num_images = template['num_images']
    if num_images != 0:
        new_slides.append({
            **slide,
            "images": images_results[image_index]
        })
        image_index += 1
lecture = {
    **result,
    "slides": new_slides
}

A stack is a Last-In-First-Out (LIFO) data structure, where the last element added is the first one to be removed. Imagine a stack of plates: you can only add or remove plates from the top. Stacks support two primary operations: 'push' to add an element and 'pop' to remove the top element.
Stacks provide several essential operations: 'push' adds an element to the top, 'pop' removes and returns the top element, 'peek' returns the top element without removing it, 'isEmpty' checks if the stack is empty, and 'isFull' checks if the stack has reached its capacity.
When considering a stack as an Abstract Data Type (ADT), we focus on the operations it provides rather than the specific implementation details. This abstraction allows us to use stacks without worrying about how they are internally implemented, providing a user-centric view of the data structure.
Stacks have numerous applications in programming, including: 1) Depth-First Search (DFS) algorithms for traversing tree and graph struct

In [7]:
lecture

{'title': 'Stacks in Programming',
 'description': 'An exploration of stacks, their properties, and applications in programming.',
 'slides': [{'title': 'Stacks in Programming',
   'template_id': 4,
   'texts': [],
   'speaker_notes': 'Welcome to the lecture on stacks in programming. Today, we will delve into the world of stacks, exploring their fundamental characteristics, operations, and diverse applications in the realm of computer science.',
   'images': []},
  {'title': 'What is a Stack?',
   'template_id': 0,
   'texts': ['LIFO data structure',
    'Push and pop operations',
    'Like a stack of plates'],
   'image': 'stack_image.jpg',
   'speaker_notes': "A stack is a Last-In-First-Out (LIFO) data structure, where the last element added is the first one to be removed. Imagine a stack of plates: you can only add or remove plates from the top. Stacks support two primary operations: 'push' to add an element and 'pop' to remove the top element.",
   'images': [{'src': 'https://media